# check out duplicate affiliation

In [2]:
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import fuzzywuzzy
from fuzzywuzzy import fuzz
import itertools
from itertools import combinations
from joblib import Parallel, delayed

/Users/yihuiwang/Envs/ivy/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#count cpu
import psutil
print(psutil.cpu_count())

4


In [4]:
engine = create_engine('postgresql://yihuiwang:@localhost:5432/ais')

In [5]:
sql_cmd = "select * from aff_info"
df = pd.read_sql(sql = sql_cmd, con = engine)

In [6]:
df_aff = df['affiliation-name']

#### generating conbinations

In [8]:
list0 = df_aff.tolist()

In [9]:
list_aff_pair = list(combinations(list0,2))

In [10]:
len(list_aff_pair)

41391351

#### split into multiple sub-arrays (e.g. 2100)

In [10]:
list_split = np.array_split(list_aff_pair, 2100)

In [11]:
def fun(x):
    aff_pair_score = pd.DataFrame(columns=('Affiliation1','Affiliation2','Ratio','Token_Sort_Ratio'))
    for entry in x:
        str1 = entry[0]
        str2 = entry[1]
        Ratio = fuzz.ratio(str1.lower(),str2.lower())
        Token_Sort_Ratio = fuzz.token_sort_ratio(str1, str2)
        aff_pair_score = aff_pair_score.append(pd.DataFrame({'Affiliation1':[str1],'Affiliation2':[str2],'Ratio':[Ratio],'Token_Sort_Ratio':[Token_Sort_Ratio]}),ignore_index=True)
    return aff_pair_score

In [ ]:
y = Parallel(n_jobs= 3)(delayed(fun)(list_split[i])for i in range(2100))

#### select score above 85

In [ ]:
aff_select_score = pd.DataFrame(columns=('Affiliation1','Affiliation2','Ratio','Token_Sort_Ratio'))
for index in range(300):
    df_in = y[index]
    aff_85 = df_in.loc[(df_in['Ratio']> 85) | (df_in['Token_Sort_Ratio']>85)]
    aff_select_score = aff_select.append(aff_85)

In [ ]:
aff_select_score

# 300000个尝试

In [33]:
test_aff_pair = list_aff_pair[0:300000]

In [34]:
list2 = np.array_split(test_aff_pair, 300)

In [13]:
list2

[array([['McGill University', 'Xidian University'],
        ['McGill University', 'Brunel University London'],
        ['McGill University', 'University of Massachusetts Dartmouth'],
        ...,
        ['Xidian University', 'Nove de Julho University'],
        ['Xidian University',
         'The Office of The Deputy Prime Minister (Directgov Home and Community Franchise Team'],
        ['Xidian University', 'Sahlgrenska Universitetssjukhuset']],
       dtype='<U118'), array([['Xidian University', 'Inotera Memories'],
        ['Xidian University', "Kenny's Bookshop and Art Galleries Ltd."],
        ['Xidian University', 'Bayer Corporation, USA'],
        ...,
        ['University of Massachusetts Dartmouth', 'SKIM Group'],
        ['University of Massachusetts Dartmouth',
         'King Abdullah Petroleum Studies and Research Center'],
        ['University of Massachusetts Dartmouth',
         'Daishowa-Marubeni International']], dtype='<U118'), array([['University of Massachusetts Da

In [14]:
def fun(x):
    aff_pair_score = pd.DataFrame(columns=('Affiliation1','Affiliation2','Ratio','Token_Sort_Ratio'))
    for entry in x:
        str1 = entry[0]
        str2 = entry[1]
        Ratio = fuzz.ratio(str1.lower(),str2.lower())
        Token_Sort_Ratio = fuzz.token_sort_ratio(str1, str2)
        aff_pair_score = aff_pair_score.append(pd.DataFrame({'Affiliation1':[str1],'Affiliation2':[str2],'Ratio':[Ratio],'Token_Sort_Ratio':[Token_Sort_Ratio]}),ignore_index=True)
    return aff_pair_score

In [35]:
%%time
y = Parallel(n_jobs= 3)(delayed(fun)(list2[i])for i in range(300))

CPU times: user 5.78 s, sys: 962 ms, total: 6.74 s
Wall time: 6min 22s


In [52]:
aff_select = pd.DataFrame(columns=('Affiliation1','Affiliation2','Ratio','Token_Sort_Ratio'))
for index in range(300):
    df_in = y[index]
    aff_a = df_in.loc[(df_in['Ratio']> 85) | (df_in['Token_Sort_Ratio']>85)]
    aff_select = aff_select.append(aff_a)

In [53]:
aff_select

,Affiliation1,Affiliation2,Ratio,Token_Sort_Ratio
88,McGill University,Magill University,94,94
331,McGill University,Comilla University,86,86
351,Xidian University,Indiana University,91,57
931,Xidian University,Xiamen University,88,88
814,Xidian University,Dalian University,88,59
286,Xidian University,Diwan University,91,61
634,Xidian University,Iman University,88,62
349,Brunel University London,University of Brunel,55,86
588,University of Massachusetts Dartmouth,University of Massachusetts and ITT,86,86
728,"University of Maryland, Baltimore County","University of Maryland, Baltimore",90,90
